## JSON import, info

In [23]:
import pandas as pd
import json

# Example JSON string
with open('predictions_checkpoint_1800.json', 'r') as f:
    data = json.load(f)


# Convert to DataFrame
df = pd.DataFrame(data)

In [22]:
df.columns

Index(['company_id', 'year', 'actual_label', 'predicted_direction',
       'magnitude', 'certainty', 'reason', 'is_match', 'full_model_response'],
      dtype='object')

In [63]:
print(df['predicted_direction'].value_counts())

predicted_direction
error                                   560
increase                                559
decrease                                398
<increase/decrease>                     122
moderate increase                        40
uncertain                                39
possible increase                        24
possible decrease                        12
moderate                                  8
slight increase                           6
steady                                    4
stable                                    3
stabilize                                 2
moderate decrease                         2
decreased                                 2
possible stabilization                    1
possibly decrease                         1
stabilization                             1
unknown                                   1
maintained                                1
decremented                               1
increased                                 1
slight      

In [3]:
(df == 'error').sum()

company_id               0
year                     0
actual_label             0
predicted_direction    560
magnitude                0
certainty                0
reason                   0
is_match                 0
full_model_response      0
dtype: int64

In [ ]:

print(df['full_model_response'][0].replace('\\n', '\n').replace('\\e', ' '))

system
You are a helpful assistant.
user
Based on the following financial statements, assess whether EPS will increase or decrease in the next year:

Below is the Balance Sheet:
------------------------------------------------------------------------------------------
| Account Items                                                  | 1971 | 1972 |
------------------------------------------------------------------------------------------
| Total Asset                                                  | 29.330 | 19.907 |
| Current Assets                                               | 19.688 | 11.326 |
| Current Liabilities                                          | 8.507 | 3.990 |
| Cash and Short-Term Investments                              | 2.557 | 2.027 |
| Receivables                                                  | 8.398 | 4.186 |
| Inventories                                                  | 8.187 | 4.743 |
| Other Current Assets                                         | 0.54

## My code begins

In [ ]:
# TODO i need to take the json that's given, just extract the prev "full_model_response" of the json and save it to a new one-- This is done
# TODO apply regex and just get the part after the second "assistant" -- this is done
# TODO parse the result to make it into a new json. like this: --- Doing but it's failing
# return {
#                 "direction": data.get("Direction of EPS Change", "Unknown"),
#                 "magnitude": data.get("Magnitude of Change", "Unknown"),
#                 "certainty": data.get("Certainty of Assessment", "Unknown"),
#                 "reason": data.get("Reason", "No reason provided."),
#                 "full_response": output # Always return the full raw output
#             }
# TODO make sure it's not giving "error" anywhere
# TODO take the actual_label,predicted_direction columns of the json and print the F1 score.


In [ ]:
## TODO 1 & 2 -- WORKING!!
import re
import json

# Regex pattern to extract text after second "assistant"
PATTERN = re.compile(r'(?si).*?assistant.*?assistant(.*)', re.DOTALL)

def extract_after_second_assistant(text):
    match = PATTERN.search(text)
    if match:
        return match.group(1).strip()
    return ""  # Return empty string if no match found

def clean_full_model_responses(json_data):
    for item in json_data:
        if "full_model_response" in item:
            # If full_model_response is a dict, convert to string
            if isinstance(item["full_model_response"], dict):
                fm_str = json.dumps(item["full_model_response"])
            else:
                fm_str = item["full_model_response"]

            # Apply regex
            cleaned_text = extract_after_second_assistant(fm_str)
            item["full_model_response"] = cleaned_text

    return json_data

def process_json_data(json_data):
    for item in json_data:
        if "full_model_response" in item:
            # If full_model_response is a dict, convert to string
            if isinstance(item["full_model_response"], dict):
                fm_str = json.dumps(item["full_model_response"])
            else:
                fm_str = item["full_model_response"]

            # Apply regex
            cleaned_text = extract_after_second_assistant(fm_str)
            item["full_model_response"] = cleaned_text

    return json_data



In [ ]:
## TODO 1 & 2 -- WORKING!!
# Example usage with your sample data point
with open('predictions_checkpoint_1800.json', 'r') as f:
    data = json.load(f)
all_results = process_json_data(data)

with open("updated_json.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

# print(json.dumps(updated_json, indent=2))

In [ ]:

# def parse_response(output: str):
#     print("PARSING RESPONSE")

#     # Remove common markdown/code formatting
#     cleaned_output = output.replace("```json", "").replace("```", "").strip()

#     # Fix bad escape sequences (like \e), replace with space
#     cleaned_output = re.sub(r'\\[a-zA-Z]', ' ', cleaned_output)

#     # Extract any JSON-looking substring: { ... }
#     json_match = re.search(r'\{[^{}]*"direction[^{}]*\}', cleaned_output, re.DOTALL | re.IGNORECASE)

#     if json_match:
#         json_str = json_match.group(0)

#         # Remove trailing commas
#         json_str = re.sub(r',\s*([}\]])', r'\1', json_str)

#         try:
#             data = json.loads(json_str)

#             # Normalize keys
#             data = {k.lower().replace(" ", "_"): v for k, v in data.items()}

#             return {
#                 "direction": data.get("direction_of_eps_change", data.get("direction", "Unknown")).lower(),
#                 "magnitude": data.get("magnitude_of_change", data.get("magnitude", "Unknown")),
#                 "certainty": data.get("certainty_of_assessment", data.get("certainty", "Unknown")),
#                 "reason": data.get("reason", "No reason provided."),
#                 "full_response": cleaned_output
#             }

#         except json.JSONDecodeError as e:
#             print(f"⚠️ JSON parsing failed: {str(e)}")

#     # Fallback regex-based extraction
#     print("❗ No valid JSON block found; using basic regex fallback.")

#     def extract_value(field, text):
#         match = re.search(fr'{field}[^:\w]*:[^"\w]*(["\w][^,}}"]*)', text, re.IGNORECASE)
#         if match:
#             return match.group(1).strip().strip('"')
#         return "Unknown"

#     direction = extract_value(r'direction|direction_of_eps_change', cleaned_output)
#     magnitude = extract_value(r'magnitude|magnitude_of_change', cleaned_output)
#     certainty = extract_value(r'certainty|certainty_of_assessment', cleaned_output)
#     reason = extract_value(r'reason', cleaned_output)

#     return {
#         "direction": direction,
#         "magnitude": magnitude,
#         "certainty": certainty,
#         "reason": reason,
#         "full_response": cleaned_output
#     }


In [56]:
import re
import json

def parse_response(output: str):
    print("PARSING RESPONSE")

    if not isinstance(output, str):
        try:
            output = str(output)
        except:
            return {
                "direction": "Unknown",
                "magnitude": "Unknown",
                "certainty": "Unknown",
                "reason": "No reason provided.",
                "full_response": ""
            }

    # Try to find JSON block containing prediction
    match = re.search(r'\{.*"direction_of_eps_change".*?\}', output, re.DOTALL)
    if not match:
        print("❌ No JSON found in model response.")
        return {
            "direction": "Unknown",
            "magnitude": "Unknown",
            "certainty": "Unknown",
            "reason": "No reason provided.",
            "full_response": output
        }

    json_str = match.group(0)

    # Remove bad escape patterns (like \e) from the JSON string before parsing
    json_str = re.sub(r'\\e', '', json_str)

    # Fix any trailing commas
    json_str = re.sub(r',\s*([}\]])', r'\1', json_str)

    try:
        data = json.loads(json_str)

        return {
            "direction": str(data.get("direction_of_eps_change", "Unknown")).strip().lower(),
            "magnitude": str(data.get("magnitude_of_change", "Unknown")).strip(),
            "certainty": str(data.get("certainty_of_assessment", "Unknown")).strip(),
            "reason": str(data.get("reason", "No reason provided.")).strip().replace("\\\\", "").replace("\\", ""),
            "full_response": output
        }

    except json.JSONDecodeError as e:
        print(f"⚠️ Failed to parse JSON: {str(e)}")
        return {
            "direction": "Unknown",
            "magnitude": "Unknown",
            "certainty": "Unknown",
            "reason": "JSON decode error.",
            "full_response": output
        }

In [57]:
# --- Evaluation Logic ---
def evaluate_predictions(updated_json, output_file="eval_json.json"):
    all_results = []

    for item in updated_json:
        company_id = item.get("company_id", "Unknown")
        year = item.get("year", "Unknown")
        actual_label = str(item.get("actual_label", "Unknown")).strip().lower()

        if "full_model_response" not in item:
            print(f"Skipping entry {company_id} - missing full_model_response")
            continue

        raw_full_response = item["full_model_response"]
        cleaned_full_response = raw_full_response.replace('\n}\n```', '').rstrip('`').strip()
        parsed_output = parse_response(cleaned_full_response)
        predicted_direction = str(parsed_output.get("direction", "Unknown")).strip().lower()

        is_match = predicted_direction in ["increase", "decrease"] and predicted_direction == actual_label

        result_entry = {
            "company_id": company_id,
            "year": year,
            "actual_label": actual_label,
            "predicted_direction": predicted_direction,
            "magnitude": parsed_output.get("magnitude", "Unknown"),
            "certainty": parsed_output.get("certainty", "Unknown"),
            "reason": parsed_output.get("reason", "Unknown"),
            "is_match": is_match,
            "full_model_response": cleaned_full_response
        }

        all_results.append(result_entry)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    print(f"✅ Results saved to '{output_file}'")



In [58]:
sample_output = '''### Step-by-Step Reasoning\n\n#### 1. Analyzing the Balance Sheet\n\n**Year-over-year percentage changes:**\n\n* **Total Assets**: Decreased from $29.330M to $19.907M (-31%)\n    * *Current Assets*: Shrank significantly, especially in receivables (\\(41\\%\\)) and inventories (\\(44\\%\\)).\n    * *Non-current Assets*: Decreases were observed across property plant equipment net (-\\(8.317\\)M), investments & advances equities (-\\(0.145\\)M), intangibles assets (-\\$0.028M).\n  \n**Key Insights/Trends:** \nThe total asset base notably shrunk due mainly to reductions in current assets, with significant declines noted in receivables and inventories. This suggests potential lower sales activity or tightened inventory management.\n\n#### 2. Performing a Cash Flow Analysis\n\n**Liquidity and Cash Movement:**\nFrom the decrease in total assets and particularly current assets alongside an overall decline in sales, it can be inferred that operating activities may have led to reduced inflows compared to outflows potentially impacting working capital negatively. However, without specific cash flow details, one cannot conclusively determine liquidity position.\n\n#### 3. Analyzing the Income Statement\n\n**Year-over-year percentage changes:**\n* **Sales**: Declined sharply from \\( \\$47.335M \\) to \\( \\$34.362M \\(-27\\%\\))\n    * **Costs of Good Sold**: Followed suit decreasing but more steeply than revenues.\n    \n**Key Insights/Trends:**\nRevenue has dropped considerably while cost of goods sold decreased at a higher rate, indicating possible overstocking issues or markdowns/sales strategies which could impact gross margins positively now.\n\n#### 4. Financial Ratio Analysis\n\nCalculations (are not provided here, yet let’s assume typical metrics like ROE, DSO, COGS turnover etc. would reveal declining performance given the scenario described so far.\n\n**Insights/Trends:**\nROA might shrink, leading to a negative signal regarding earnings capacity relative to its assets investment. DSO increased due to receivable reduction implying slow payment cycles possibly from customer base contraction. \n\n#### 5. Final Summary and Future Outlook\n\nGiven the substantial drop in both revenue generation and assets accumulation coupled with high costs associated with these decreases, there seems to be pressure on profitability through reduced margins and potential erosion of shareholder equity if dividend payments remain steady or increase slightly.\n\n#### 6. Predicting EPS Change for Next Year\n\nAssuming similar dynamics continue into fiscal '73,\n\n**Prediction Result in JSON Format:**\n```json\n{\n  \"direction_of_eps_change\": \"Decrease\",\n  \"magnitude_of_change\": \"Significant\",\n  \"certainty_of_assessment\": \"Medium\",\n  \"reason\": \"\\\\eSince\\\\e\\\\ sales\\\\ehave\\\\edeclined\\\\esharply\\\\ewith\\\\ecosts\\\\egood\\\\sold\\\\edropping\\\\emore\\\\estep\\\\ethan\\\\erevenues,\\\\eindicating\\\\efocused\\\\enventory\\\\emanagement\\\\ehowever,\\theffects\\\\eon\\\\egross\\\\emerits\\\\emargins\\\\emeet\\\\eargative\\\\ensignal,\\\\ewhile\\\\elower\\\\ebalance\\\\eshirt\\\\eassets\\\\emight\\\\eelevate\\\\ecntrl\\\\eerisk\\\\eto\\\\eshrinking\\\\eedline,\\\\ean\\\\eduction\\\\ein\\\\ebottom-line\\\\efficiency\\\\ecould\\\\eaffect\\\\eearnings-per-share\\\\enevitably\\\\efavorably.\"\n}\n```\nThis result assumes no improvements in operational efficiency, market recovery, or change in financial policy affecting expenses or dividends. Real predictions should consider such factors for accuracy.
'''

In [59]:

# --- Usage ---
with open('predictions_checkpoint_1800.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

cleaned_data = clean_full_model_responses(raw_data)

with open("updated_json.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)

# Load the cleaned version again
with open("updated_json.json", "r", encoding="utf-8") as f:
    updated_json = json.load(f)
    
evaluate_predictions(updated_json, "eval_json.json")

PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESPONSE
❌ No JSON found in model response.
PARSING RESP

## IDK yet

In [17]:

def process_json_data(updated_json, output_file="eval_json.json"):
    all_results = []

    for item in updated_json:
        company_id = item.get("company_id", "Unknown")
        year = item.get("year", "Unknown")
        actual_label = str(item.get("actual_label", "Unknown")).strip().lower()

        if "full_model_response" not in item:
            print(f"Skipping entry {company_id} - missing full_model_response")
            continue

        raw_full_response = item["full_model_response"]

        # Step 1: Clean the raw response
        cleaned_full_response = raw_full_response.replace('\n}\n```', '').rstrip('`').strip()

        # Step 2: Parse the cleaned response
        parsed_output = parse_response(cleaned_full_response)

        predicted_direction = str(parsed_output.get("direction", "Unknown")).strip().lower()

        # Step 3: Determine match
        is_match = False
        if predicted_direction in ["increase", "decrease"]:
            is_match = (predicted_direction == actual_label)

        # Step 4: Build result entry
        result_entry = {
            "company_id": company_id,
            "year": year,
            "actual_label": actual_label,
            "predicted_direction": predicted_direction,
            "magnitude": parsed_output.get("magnitude", "Unknown"),
            "certainty": parsed_output.get("certainty", "Unknown"),
            "reason": parsed_output.get("reason", "Unknown"),
            "is_match": is_match,
            "full_model_response": cleaned_full_response  # Use cleaned version
        }

        all_results.append(result_entry)

    # Step 5: Save results to eval_json.json
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    print(f"✅ Results saved to '{output_file}'")

In [19]:
with open('predictions_checkpoint_1800.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

cleaned_data = clean_full_model_responses(raw_data)

with open("updated_json.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)

# Load the cleaned version again
with open("updated_json.json", "r", encoding="utf-8") as f:
    updated_json = json.load(f)

evaluate_predictions(updated_json, "eval_json.json")

PARSING RESPONSE


error: unknown extension ?R at position 13